## Mounting notebook to Azure

In [0]:
dbutils.widgets.text("accountName", "", "Account Name")
dbutils.widgets.text("accountKey", "", "Account Key")

In [0]:
#Get values entered into widgets
accountName = dbutils.widgets.get("accountName")
accountKey = dbutils.widgets.get("accountKey")

In [0]:
#mounting blob storage
if not any(mount.mountPoint == '/mnt/smartmeterdata' for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(
        source = "wasbs://smartmeterdata@" + accountName + ".blob.core.windows.net/smartmeterdata",
        mount_point = "/mnt/smartmeterdata",
        extra_configs = {"fs.azure.account.key." + accountName + ".blob.core.windows.net": accountKey}
    )

## Extract Data from Azure Blob Storage

In [0]:
#Inspect file structure
display(dbutils.fs.ls("/mnt/smartmeterdata/"))

path name size modificationTime dbfs:/mnt/smartmeterdata/2023/ 2023/ 0 0

In [0]:
#create a dataframe
df = spark.read.options(header='true', inferSchema='true').csv("dbfs:/mnt/smartmeterdata/*/*/*/*.csv", header=True)
print(df.dtypes)

[('id', 'string'), ('time', 'string'), ('temp', 'int'), ('voltage', 'double'), ('EventProcessedUtcTime', 'timestamp'), ('PartitionId', 'int'), ('EventEnqueuedUtcTime', 'timestamp'), ('IoTHub', 'string')]


In [0]:
df = df.withColumnRenamed('voltage','PowerConsumption')
display(df)

Data extracted successfully
Below is Collected Data Count

In [0]:
df.count()
df.show(10)

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-4431128920746675>:2
      1 df.count()
----> 2 df.show(10)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         error_class="NOT_A_BOOLEAN",
    916         message_parameters={"arg_name": "vertical", "arg_type": type(vertical).__name__},
    917     )
    919 if isinstance(truncate, bool) and truncate:
--> 920     print(self._jdf.showString(n, 20, ver

## Data Transformation
Converting Timestamp to meaning data

In [0]:
from pyspark.sql.functions import *
#Converting the Timestamp
df = df.withColumn('date', to_timestamp(col('time'), 'yyyy-MM-dd\'T\'HH:mm:ss.SSSSSSS\'Z\''))
df = df.withColumn('year', year(col('date')))
df = df.withColumn('month', month(col('date')))
df = df.withColumn('day', dayofmonth(col('date')))
df = df.withColumn('weekday', date_format(col('date'), 'EEEE'))
df = df.withColumn('weekofyear', weekofyear(col('date')))
df = df.withColumn('hour', hour(col('date')))
df = df.withColumn('minute', minute(col('date')))

df.show(3)


+-------+--------------------+----+-------------------+---------------------+-----------+--------------------+------+--------------------+----+-----+---+--------+----------+----+------+
|     id|                time|temp|   PowerConsumption|EventProcessedUtcTime|PartitionId|EventEnqueuedUtcTime|IoTHub|                date|year|month|day| weekday|weekofyear|hour|minute|
+-------+--------------------+----+-------------------+---------------------+-----------+--------------------+------+--------------------+----+-----+---+--------+----------+----+------+
|Device0|2023-04-20T11:22:...|  71| 0.7870723111932503| 2023-04-20 11:23:...|          3|2023-04-20 11:23:...|Record|2023-04-20 11:22:...|2023|    4| 20|Thursday|        16|  11|    22|
|Device4|2023-04-20T11:23:...|  60| 0.8604929382700905| 2023-04-20 11:23:...|          3|2023-04-20 11:23:...|Record|2023-04-20 11:23:...|2023|    4| 20|Thursday|        16|  11|    23|
|Device3|2023-04-20T11:23:...|  56|0.34306107537684083| 2023-04-20 11:

In [0]:
#Dropping unwanted columns
df = df.drop("time","EventProcessedUtcTime","PartitionId","EventEnqueuedUtcTime","IoTHub")

## Saving Tidy Dataset as Table

In [0]:
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("SmartMeters")

In [0]:
%sql
select * from smartmeters

In [0]:
#removing outlier
#df = df.filter((col('temp') > 0) & (col('temp') < 100))

In [0]:
data = df

## Building Forecasting Model

In [0]:
import matplotlib.pyplot as plt
from prophet import Prophet
from pyspark.sql.functions import to_date, struct
import pandas as pd

In [0]:

# Take relevant information for the model
data = data.withColumnRenamed('date','ds')
data = data.drop('year', 'month', 'day', 'weekday', 'weekofyear', 'hour', 'minute','date')


#Forecast Power Consumption for each Device

In [0]:
#sorting datasets as per device
unique_devices = sorted(data.select("id").distinct().rdd.flatMap(lambda x: x).collect())
figs = []
for device in unique_devices:
    new_df = data.filter(data.id == device)
    new_df = new_df.withColumnRenamed('PowerConsumption','y')
    fdf = new_df['ds','y']
    fpf = fdf.toPandas()
    fpf.set_index('ds',inplace=True)
    #Resampling data from seconds to hours
    fpf = fpf.resample('1H').mean()
    #Filling NaN values by Mean
    fpf.fillna(fpf.mean(), inplace=True)
    fpf = fpf.reset_index().rename(columns={'ds': 'ds', 'y': 'y'})
    prophet_model = Prophet()
    prophet_model.fit(fpf)
    future = prophet_model.make_future_dataframe(periods=1440, freq='min')
    forecast = prophet_model.predict(future)
    fig = prophet_model.plot(forecast)
    fig.suptitle(f"Forecast for {device}")
    figs.append(fig)

plt.show()

